### **Система перевода речи в реальном времени**


Функционал:
  - Распознает речь на входном языке
  - Переводит на целевой язык
  - Синтезирует речь на целевом языке
  - Сохраняет характеристики голоса говорящего

# Установка и импорт

In [ ]:
!pip install gradio
!pip install transformers
!pip install translators
!pip install -q TTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.5 MB/s eta 0:00:00
   ━

In [ ]:
import translators as ts # Переводчик

from IPython.display import HTML # для активации JavaScript

import gradio as gr # Интерфейс
from transformers import pipeline # Запуск модели с HF

from TTS.api import TTS  # API для Text-to-Speech

# Архитектура модели

In [ ]:
# @title Включение микрофона
js='''
<script>
navigator.mediaDevices.getUserMedia({audio: true})
</script>
'''
display(HTML(js))

In [ ]:
# @title Константы
class RealTimeTranslator():
    def __init__(self):
        # Для выбора языка
        self.langs=("en", "es", "fr","de","it","pt","pl","tr","ru","nl","cs",
                    "ar","zh-cn","ja","hu","ko","hi")

        # Выбор языка для человека (удобочитаемый)
        self.lang_for_user ="""английский (en) испанский (es) французский (fr) немецкий (de)
                        итальянский (it) португальский (pt) польский (pl) турецкий (tr)
                        русский (ru) голландский (nl) чешский (cs) арабский (ar) китайский (zh-cn)
                        японский (ja) венгерский (hu) корейский (ko) хинди (hi)"""

        self.whisper = pipeline("automatic-speech-recognition","openai/whisper-large-v3-turbo")

        self.target_audio_name = 'output.wav' # Целевое название файла

        self.tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=False) # Модель клонирования

    def choice_language(self, lang):
        return True if lang in self.langs else False

    def speech_to_text(self, speech, lang):
        return self.whisper(speech)['text']

    def translate_to_text(self, text, lang):
        return ts.translate_text(text, to_language=lang)

    def translate_speech(self, audio, target_text, lang):
        self.tts.tts_to_file(text=target_text,              # Целевой текст
                        file_path=self.target_audio_name,   # Путь до целевого аудио
                        speaker_wav=audio, # Исходное аудио
                        language=lang           # Целевой язык
                        )
        return self.target_audio_name

rtt = RealTimeTranslator()

# Интерфейс

In [ ]:
def gr_choice_language(lang):
    choice = rtt.choice_language(lang)
    if choice:
        return gr.Textbox(interactive=False)
    else:
        gr.Warning("Неправильный ввод!")

def gr_speech_to_text(speech, lang):
    if lang != '':
        return rtt.speech_to_text(speech, lang)
    else:
        gr.Warning('Введите целевой язык!')

def gr_translate_to_text(text, lang):
    if lang != '':
        return rtt.translate_to_text(text, lang)

def gr_translate_speech(audio, target_text, lang):
    if lang !='':
        return rtt.translate_speech(audio, target_text, lang)




with gr.Blocks() as demo:


    with gr.Row():
        with gr.Column():
            language = gr.Textbox(label='Целевой язык аудиофайла',
                                  info=rtt.lang_for_user)

            audio = gr.Audio(sources="microphone", type="filepath")

        with gr.Column():
            sourse_text = gr.Textbox(label='Текст оригинальной записи', interactive=False)
            target_text = gr.Textbox(label='Текст целевой записи', interactive=False)
            target_audio = gr.Audio(label='Аудио на целевом языке', interactive=False)

    language.submit(fn=gr_choice_language,
                    inputs=language,
                    outputs=language)

    audio.stop_recording(fn=gr_speech_to_text, inputs=[audio, language], outputs=sourse_text)\
     .then(fn=gr_translate_to_text, inputs=[sourse_text, language], outputs=target_text)\
     .then(fn=gr_translate_speech, inputs=[audio, target_text, language], outputs=target_audio)


demo.launch()